In [1]:
import copy
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DynamicCache, StaticCache

model_id = f"hugging-quants/Meta-Llama-3.1-8B-BNB-NF4-BF16"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="cuda",
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# Init StaticCache with big enough max-length (1024 tokens for the below example)
# You can also init a DynamicCache, if that suits you better
# prompt_cache = StaticCache(config=model.config, batch_size=1, max_cache_len=1024, device="cuda", dtype=torch.bfloat16)

INITIAL_PROMPT = "You are a helpful assistant. " * 50
INITIAL_PROMPT_TOKS = tokenizer.encode(INITIAL_PROMPT)
# This is the common prompt cached, we need to run forward without grad to be abel to copy
with torch.no_grad():
    prompt_cache = model(
        input_ids=torch.tensor(INITIAL_PROMPT_TOKS).unsqueeze(0).to("cuda"),
        use_cache=True,
    ).past_key_values

prompts = [
    "Help me to write a blogpost about travelling.",
    "What is the capital of France?",
] * 10
responses = []
for prompt in prompts:
    prompt_toks = tokenizer.encode(prompt)
    past_key_values = copy.deepcopy(prompt_cache)
    outputs = model.generate(
        torch.tensor(INITIAL_PROMPT_TOKS + prompt_toks).unsqueeze(0).to("cuda"),
        do_sample=False,
        past_key_values=past_key_values,
        max_new_tokens=20,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = tokenizer.batch_decode(outputs)[0]
    responses.append(response)

print(responses)

/root/cot-probing/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/cot-probing/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


IndexError: index -1 is out of bounds for dimension 0 with size 0

In [3]:
type(prompt_cache)

tuple